## Using the cloned open-data
Assuming you have cloned the open-data using git you can also get a list of files with glob.
If you are not using git, comment this out



In [ ]:
# change the path of STATSBOMB_DATA to the location of you open-data
# STATSBOMB_DATA = os.path.join('..', '..', '..', 'open-data','data')
# event_links = glob.glob(os.path.join(STATSBOMB_DATA, 'events', '**', '*.json'),recursive=True)
# lineup_links = glob.glob(os.path.join(STATSBOMB_DATA, 'lineups', '**', '*.json'),recursive=True)
# match_links = glob.glob(os.path.join(STATSBOMB_DATA, 'matches', '**', '*.json'),recursive=True)
# COMPETITION_PATH = os.path.join(STATSBOMB_DATA, 'competitions.json')

# print('Number of event files:',len(event_links))
# print('Number of lineup files:', len(lineup_links))
# print('Number of match files:', len(match_links))

## Setup some destination folders
Now we need to setup some folders to store the dataframes.
We are going to setup the following directory structure

| data <- top level directory to store the combined dataframes
| ├── event_raw <- Folder for event data
| ├── related_raw <- Folder for info on how events are connected
| ├── freeze_raw <- Folder for the individual shot freeze frames
| ├── tactic_raw <-Folder for the lineup tactics
| ├── lineup_raw <- Folder for the lineup info

I am saving the dataframes as parquet files as they are small and load rapidly
(see here for more info https://ursalabs.org/blog/2019-10-columnar-perf/).



In [5]:
# Amend this path to where you want to store the data
DATA_FOLDER = os.path.join(r'C:\........')

# make the directory structure
for folder in ['event_raw', 'related_raw', 'freeze_raw', 'tactic_raw', 'lineup_raw']:
    path = os.path.join(DATA_FOLDER, folder)
    if not os.path.exists(path):
        os.mkdir(path)

## Competition data
Get the competition data as a dataframe as save as parquet file



In [6]:
df_competition = sbapi.read_competition(COMPETITION_PATH, warn=False)
# note there is a slight loss of data quality with timestamps,
# but these aren't relevant for analysis
# pandas has nanoseconds, which aren't supported in parquet (supports milliseconds)
df_competition.to_parquet(os.path.join(DATA_FOLDER, 'competition.parquet'),
                          allow_truncated_timestamps=True)
df_competition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   competition_id      37 non-null     int64         
 1   season_id           37 non-null     int64         
 2   country_name        37 non-null     object        
 3   competition_name    37 non-null     object        
 4   competition_gender  37 non-null     object        
 5   season_name         37 non-null     object        
 6   match_updated       37 non-null     datetime64[ns]
 7   match_available     37 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 2.4+ KB


## Keep a copy of the old match data
We are going to use this to compare to the new match file and check for updates.



In [7]:
match_path = os.path.join(DATA_FOLDER, 'match.parquet')
if os.path.exists(match_path):
    df_match_copy = pd.read_parquet(match_path).copy()
    UPDATE_FILES = True
else:
    UPDATE_FILES = False

## Match data
Get the match data as a dataframe and save as parquet.
Note there is a mismatch between the length of this file
and the number of event files because some event files don't have match data.



In [27]:
df_match.sort_values('match_id').head()

,match_id,match_date,kick_off,home_score,away_score,match_status_360,last_updated,last_updated_360,match_week,competition_id,...,home_team_managers_dob,home_team_managers_country_id,home_team_managers_country_name,away_team_managers_id,away_team_managers_name,away_team_managers_nickname,away_team_managers_dob,away_team_managers_country_id,away_team_managers_country_name,metadata_xy_fidelity_version
0,7430,2018-04-15,2018-04-15 01:00:00,2,4,unscheduled,2020-07-29 05:00:00,NaN,3,49,...,1959-09-22,241.0,United States of America,127.0,Paul Riley,None,NaT,68.0,England,NaN
1,7443,2018-05-05,2018-05-05 21:30:00,2,3,unscheduled,2020-07-29 05:00:00,NaN,6,49,...,1986-08-08,241.0,United States of America,141.0,Vlatko Andonovski,None,NaT,241.0,United States of America,NaN
3,7444,2018-05-06,2018-05-06 21:00:00,1,1,unscheduled,2020-07-29 05:00:00,NaN,6,49,...,NaT,68.0,England,142.0,Rory Dames,None,NaT,241.0,United States of America,NaN
2,7445,2018-05-06,2018-05-06 03:00:00,2,0,unscheduled,2020-07-29 05:00:00,NaN,6,49,...,NaT,68.0,England,131.0,James Gabarra,None,1959-09-22,241.0,United States of America,NaN
4,7451,2018-05-13,2018-05-13 01:00:00,1,0,unscheduled,2020-07-29 05:00:00,NaN,7,49,...,NaT,68.0,England,131.0,James Gabarra,None,1959-09-22,241.0,United States of America,NaN


In [11]:
match_dfs = [sbapi.read_match(file, warn=False) for file in match_links]
df_match = pd.concat(match_dfs)
# again there is a slight loss of quality when saving timestamps, but only relevant for last_updated
df_match.to_parquet(os.path.join(DATA_FOLDER, 'match.parquet'),
                    allow_truncated_timestamps=True)
df_match.info()

Skipping https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/16/42.json: empty json
Skipping https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/16/76.json: empty json
<class 'pandas.core.frame.DataFrame'>
Int64Index: 879 entries, 0 to 51
Data columns (total 50 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   match_id                         879 non-null    int64         
 1   match_date                       879 non-null    datetime64[ns]
 2   kick_off                         877 non-null    datetime64[ns]
 3   home_score                       879 non-null    int64         
 4   away_score                       879 non-null    int64         
 5   match_status_360                 878 non-null    object        
 6   last_updated                     879 non-null    datetime64[ns]
 7   last_updated_360                 0 non-null      float64  

## Get a list of games which have been updated



In [19]:
if UPDATE_FILES:
    df_match_copy = (df_match[['match_id', 'last_updated']]
                     .merge(df_match_copy[['match_id', 'last_updated']],
                            how='left', suffixes=['', '_old'], on='match_id'))
    df_match_copy = df_match_copy[(df_match_copy.last_updated.dt.floor('ms') !=
                                   df_match_copy.last_updated_old.dt.floor('ms'))].copy()
    to_update = df_match_copy.match_id.unique()

    # get array of event links to update - based on whether they have been updated in the match json
    event_link_ids = [int(os.path.splitext(os.path.basename(link))[0]) for link in event_links]
    event_to_update = [link in to_update for link in event_link_ids]
    event_links = np.array(event_links)[event_to_update]

    # get array of lineup links to update -
    # based on whether they have been updated in the match jsons
    lineup_link_ids = [int(os.path.splitext(os.path.basename(link))[0]) for link in lineup_links]
    lineup_to_update = [link in to_update for link in lineup_link_ids]
    lineup_links = np.array(lineup_links)[lineup_to_update]

## Subset a few files for demo purposes
For the purposes of the demo, we will take the first five event and lineup files
Comment this out if you want the whole of the open-data



In [32]:
lineup_links = lineup_links
event_links = event_links

## Lineup data



In [35]:
# Amend this path to where you want to store the data
LINEUP_FOLDER = os.path.join(DATA_FOLDER, 'lineup_raw')
# loop through all the changed links and store as parquet files - small and fast files
for file in tqdm(lineup_links):
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        df_lineup = sbapi.read_lineup(file, warn=False)
        df_lineup.to_parquet(os.path.join(LINEUP_FOLDER, save_path))
    except ValueError:
        print('Skipping file:', file)

  0%|          | 0/890 [00:00<?, ?it/s]

Get the lineup files as a single dataframe



In [36]:
if len(lineup_links) == 0:
    print('No update')
else:
    lineup_files = glob.glob(os.path.join(LINEUP_FOLDER, '*.parquet'))
    df_lineup = pd.concat([pd.read_parquet(file) for file in lineup_files])
    df_lineup.to_parquet(os.path.join(DATA_FOLDER, 'lineup.parquet'))
    df_lineup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26794 entries, 0 to 35
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   team_id               26794 non-null  int64 
 1   team_name             26794 non-null  object
 2   match_id              26794 non-null  int64 
 3   player_id             26794 non-null  int64 
 4   player_name           26794 non-null  object
 5   player_nickname       14740 non-null  object
 6   player_jersey_number  26794 non-null  int64 
 7   player_country_id     26794 non-null  int64 
 8   player_country_name   26794 non-null  object
dtypes: int64(5), object(4)
memory usage: 2.0+ MB


## Event data
We will also loop through the first five event files.
However, the ``read_event`` function returns a dictionary of four dataframes:
'event', 'related_event', 'shot_freeze_frame' and 'tactics_lineup'.
It's possible to alter ``read_event`` to return fewer dataframes (see the API docs).



In [37]:
# loop through all the changed links and store as parquet files - small and fast files
for file in tqdm(event_links):
    save_path = f'{os.path.basename(file)[:-4]}parquet'
    try:
        dict_event = sbapi.read_event(file, warn=False)
        # save to parquet files
        # using the dictionary key to access the dataframes from the dictionary
        dict_event['event'].to_parquet(os.path.join(DATA_FOLDER, 'event_raw', save_path))
        dict_event['related_event'].to_parquet(os.path.join(DATA_FOLDER, 'related_raw', save_path))
        dict_event['shot_freeze_frame'].to_parquet(os.path.join(DATA_FOLDER, 'freeze_raw',
                                                                save_path))
        dict_event['tactics_lineup'].to_parquet(os.path.join(DATA_FOLDER, 'tactic_raw', save_path))
    except ValueError:
        print('Skipping:', file)

  0%|          | 0/890 [00:00<?, ?it/s]

## Get a list of match_ids to update



In [38]:
event_files = glob.glob(os.path.join(DATA_FOLDER, 'event_raw', '*.parquet'))
if UPDATE_FILES:
    ids_to_update = [int(os.path.splitext(os.path.basename(link))[0]) for link in event_links]


# Function to load the old dataframe (if exists) and combine with the updated parquet files
def update(directory, file_type, update_ids):
    """ Update an old DataFrame with files that have changed/ been added.

    Parameters
    ----------
    directory : path to directory containing the files
    file_type : str
        One of 'event', 'freeze', 'tatic', or related'
    update_ids : list of integers
        A list of the match ids to update

    Returns
    -------
    df : pandas.DataFrame
        An updated DataFrame with the new/changed matches.
    """
    # get a list of parquet files to add to the old dataframe
    files = glob.glob(os.path.join(directory, f'{file_type}_raw', '*.parquet'))
    files_id = [int(os.path.splitext(os.path.basename(file))[0]) for file in files]
    mask_update = [match_id in update_ids for match_id in files_id]
    files = np.array(files)[mask_update]
    # load the old dataframe, filter out changed matches and add the new parquet files
    df_old = pd.read_parquet(os.path.join(directory, f'{file_type}.parquet'))
    df_old = df_old[~df_old.match_id.isin(update_ids)]
    df_new = pd.concat([pd.read_parquet(file) for file in files])
    df_old = pd.concat([df_old, df_new])
    return df_old

Get event files as a single dataframe and save to parquet.



In [39]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_event = update(DATA_FOLDER, 'event', ids_to_update)
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, null_counts=True)
    else:
        df_event = pd.concat([pd.read_parquet(file) for file in event_files])
        df_event.to_parquet(os.path.join(DATA_FOLDER, 'event.parquet'))
        df_event.info(verbose=True, null_counts=True)

<ipython-input-39-4223d548f176>:11: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_event.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3198449 entries, 0 to 3746
Data columns (total 95 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   match_id                        3198449 non-null  int64  
 1   id                              3198449 non-null  object 
 2   index                           3198449 non-null  int64  
 3   period                          3198449 non-null  int64  
 4   timestamp_minute                3198449 non-null  int64  
 5   timestamp_second                3198449 non-null  int64  
 6   timestamp_millisecond           3198449 non-null  int64  
 7   minute                          3198449 non-null  int64  
 8   second                          3198449 non-null  int64  
 9   type_id                         3198449 non-null  int64  
 10  type_name                       3198449 non-null  object 
 11  sub_type_id                     280209 non-null   float64
 12  sub

Get shot freeze frames files as a single dataframe and save to parquet.



In [40]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_freeze = update(DATA_FOLDER, 'freeze', ids_to_update)
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info(verbose=True, null_counts=True)
    else:
        freeze_files = glob.glob(os.path.join(DATA_FOLDER, 'freeze_raw', '*.parquet'))
        df_freeze = pd.concat([pd.read_parquet(file) for file in freeze_files])
        df_freeze.to_parquet(os.path.join(DATA_FOLDER, 'freeze.parquet'))
        df_freeze.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 277829 entries, 0 to 341
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    277829 non-null  object 
 1   event_freeze_id       277829 non-null  int64  
 2   player_teammate       277829 non-null  bool   
 3   player_id             277829 non-null  int64  
 4   player_name           277829 non-null  object 
 5   player_position_id    277829 non-null  int64  
 6   player_position_name  277829 non-null  object 
 7   x                     277829 non-null  float64
 8   y                     277829 non-null  float64
 9   match_id              277829 non-null  int64  
dtypes: bool(1), float64(2), int64(4), object(3)
memory usage: 21.5+ MB


Get tactics files as a single dataframe and save to parquet.



In [41]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_tactic = update(DATA_FOLDER, 'tactic', ids_to_update)
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info(verbose=True, null_counts=True)
    else:
        tactic_files = glob.glob(os.path.join(DATA_FOLDER, 'tactic_raw', '*.parquet'))
        df_tactic = pd.concat([pd.read_parquet(file) for file in tactic_files])
        df_tactic.to_parquet(os.path.join(DATA_FOLDER, 'tactic.parquet'))
        df_tactic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36817 entries, 0 to 32
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    36817 non-null  object
 1   event_tactics_id      36817 non-null  int64 
 2   player_jersey_number  36817 non-null  int64 
 3   player_id             36817 non-null  int64 
 4   player_name           36817 non-null  object
 5   player_position_id    36817 non-null  int64 
 6   player_position_name  36817 non-null  object
 7   match_id              36817 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.5+ MB


Get related events files as a single dataframe and save to parquet.



In [42]:
if len(event_links) == 0:
    print('No update')
else:
    if UPDATE_FILES:
        df_related = update(DATA_FOLDER, 'related', ids_to_update)
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, null_counts=True)
    else:
        related_files = glob.glob(os.path.join(DATA_FOLDER, 'related_raw', '*.parquet'))
        df_related = pd.concat([pd.read_parquet(file) for file in related_files])
        df_related.to_parquet(os.path.join(DATA_FOLDER, 'related.parquet'))
        df_related.info(verbose=True, null_counts=True)

<ipython-input-42-fb457d88fc62>:12: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_related.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 6219794 entries, 0 to 7467
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype 
---  ------             --------------    ----- 
 0   id                 6219794 non-null  object
 1   id_related         6219794 non-null  object
 2   type_name          6219794 non-null  object
 3   index              6219794 non-null  int64 
 4   type_name_related  6219794 non-null  object
 5   index_related      6219794 non-null  int64 
 6   match_id           6219794 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 379.6+ MB
